In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#carregando os datasets que serão utilizados na análise
df_orders = pd.read_csv('Datasets/olist_orders_dataset.csv')
df_order_items = pd.read_csv('Datasets/olist_order_items_dataset.csv')
df_order_reviews = pd.read_csv('Datasets/olist_order_reviews_dataset.csv')